In [21]:
def get_zone_from_position(mat, x, y):
    x_min = x//3*3
    y_min = y//3*3        
    return [row[y_min:y_min+3] for row in mat[x_min:x_min+3]]

def get_rows_from_position(mat, x):
    x_min = x//3*3        
    return mat[x_min:x_min+3]

def get_cols_from_position(mat, y):
    y_min = y//3*3        
    return [row[y_min:y_min+3] for row in mat]

def get_code_from_position(x, y):
    return [round(x//3), round(y//3)]

def get_zeros(mat):
    return [[idx_r, idx_nb] for idx_r, rows in enumerate(mat) for idx_nb, nb in enumerate(rows) if nb == 0]
 
def matrix2dict(mat):
    coord_dic = dict(enumerate(dict(enumerate(row)) for row in mat))
    for r, c in coord_dic.items():
        for k, v in c.items():
            coord_dic[r][k] = [v]
    return coord_dic

def dict2matrix(coord_dic):
    mat = []
    for idx_r, idx_c in coord_dic.items():
        row = []
        for c, l_of_nb in idx_c.items():
            if len(l_of_nb) == 1:
                row.append(l_of_nb[0])
            else:
                row.append(0)
        mat.append(row)
    return mat

def is_done(mat):
    done = True
    dummy = [([0]*9) for i in range(9)]
    dic_done = dict(enumerate(dict(enumerate(row)) for row in dummy))
    for idx_r, r_of_nb in enumerate(mat):        
        for idx_c, nb in enumerate(r_of_nb):
            if dic_done[idx_r][idx_c] == 0 and nb > 0:
                dic_done[idx_r][idx_c] = nb
            else:
                done = False
                
    for k, v in dic_done.items():
        row = []
        for c, nb in v.items():
            row.append(nb)
        if len(set(row)) < 9:
            done = False
                
    return done

In [2]:
def simple_check(mat, x, y, l = list(range(1,10))):

    zone = get_zone_from_position(mat, x, y)
    current_numbers = list(set(mat[x] + [row[y] for row in mat] + [e for row in zone for e in row]))
    return [e for e in l if e not in current_numbers]

def zone_zero_check(mat, x, y, coord_dic):
    candidates = []
    c_position = coord_dic[x][y]
    zone = get_zone_from_position(mat, x, y)
    code = get_code_from_position(x, y)
    zeros = [[r_idx, c_idx]  for r_idx, r in enumerate(zone) for c_idx, c in enumerate(r) if c == 0]
    zeros.remove([x%3, y%3]) # remove the current zero
    for z in zeros:               
        candidates += coord_dic[z[0]+3*code[0]][z[1]+3*code[1]]    
    lone_candidates = [e for e in c_position if e not in candidates and e > 0]
    if lone_candidates:
        return lone_candidates
    else:
        return []
    
def row_zero_check(mat, x, y, coord_dic):
    candidates = []
    c_position = coord_dic[x][y]
    zeros = [r_idx for r_idx, r in enumerate(mat[x]) if r == 0]
    zeros.remove(y)
    for z in zeros:
        candidates += coord_dic[x][z]   
    lone_candidates = [e for e in c_position if e not in candidates and e > 0]
    if lone_candidates:
        return lone_candidates
    else:
        return []
    
def col_zero_check(mat, x, y, coord_dic):
    candidates = []
    c_position = coord_dic[x][y]
    zeros = [idx_c for idx_c, c in enumerate([row[y] for idx_r, row in enumerate(mat)]) if c == 0]
    zeros.remove(x)    
    for z in zeros:
        candidates += coord_dic[z][y]   
    lone_candidates = [e for e in c_position if e not in candidates and e > 0]
    if lone_candidates:
        return lone_candidates
    else:
        return []

# Native Python

In [8]:
def populate_dict(dic, mat):
    zeros = get_zeros(mat)
    for [x, y] in zeros:
        dic[x][y] = simple_check(mat, x, y)
    return dic

def update_dict(mat, x, y, dic, l_of_nb):
    row_idx = [r_idx  for r_idx, r in enumerate(mat[x]) if r == 0]
    row_idx.remove(y)
    col_idx = [idx_c for idx_c, c in enumerate([row[y] for idx_r, row in enumerate(mat)]) if c == 0]
    col_idx.remove(x)
    
    update = False
    
    if len(l_of_nb) == 1:
        # print("placing", l_of_nb[0], "at", x, y)
        update = True
        for ci in row_idx:        
            if l_of_nb[0] in dic[x][ci]:
                # print("removing", l_of_nb[0], "from square", x, ci)
                dic[x][ci].remove(l_of_nb[0])

        for ri in col_idx:
            if l_of_nb[0] in dic[ri][y]:
                # print("removing", l_of_nb[0], "from square", ri, y)
                dic[ri][y].remove(l_of_nb[0])
                
    if len(l_of_nb) >= 1:
        dic[x][y] = l_of_nb
                
    return dic, update  

def solve_matrix(mat, limit = 10):

    coord_dic = matrix2dict(mat)
    mat = dict2matrix(coord_dic)
    coord_dic = populate_dict(coord_dic, mat)
    mat = dict2matrix(coord_dic)

    done = False

    while not done and limit:

        update = True
        limit -= 1

        while update:

            update = False
            zeros = get_zeros(mat)
            for [x, y] in zeros:
                res = simple_check(mat, x, y)
                coord_dic, update = update_dict(mat, x, y, coord_dic, res)
                mat = dict2matrix(coord_dic)
                if update:
                    # print("simple check")
                    # print(len(get_zeros(mat)))
                    break
                res = zone_zero_check(mat, x, y, coord_dic)
                coord_dic, update = update_dict(mat, x, y, coord_dic, res)
                mat = dict2matrix(coord_dic)
                if update:
                    # print("zone zero check")
                    # print(len(get_zeros(mat)))
                    break
                res = row_zero_check(mat, x, y, coord_dic)
                coord_dic, update = update_dict(mat, x, y, coord_dic, res)
                mat = dict2matrix(coord_dic)
                if update:
                    # print("row zero check")
                    # print(len(get_zeros(mat)))
                    break
                res = col_zero_check(mat, x, y, coord_dic)
                coord_dic, update = update_dict(mat, x, y, coord_dic, res)
                mat = dict2matrix(coord_dic)
                if update:
                    # print("col zero check")
                    # print(len(get_zeros(mat)))
                    break


        done = is_done(mat)
        
        
    return mat, done

In [22]:
# Grille N° 4540 Obs - difficile

m = [([0]*9) for i in range(9)]

m[0] = [7,2,0,0,0,0,0,0,0]
m[1] = [0,3,0,5,0,0,0,0,9]
m[2] = [0,0,9,3,7,0,0,0,0]

m[3] = [4,9,0,0,0,0,0,2,1]
m[4] = [1,0,0,2,0,4,0,0,6]
m[5] = [6,8,0,0,0,0,0,4,5]

m[6] = [0,0,0,0,8,6,3,0,0]
m[7] = [3,0,0,0,0,5,0,9,0]
m[8] = [0,0,0,0,0,0,0,5,4]

m

[[7, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 5, 0, 0, 0, 0, 9],
 [0, 0, 9, 3, 7, 0, 0, 0, 0],
 [4, 9, 0, 0, 0, 0, 0, 2, 1],
 [1, 0, 0, 2, 0, 4, 0, 0, 6],
 [6, 8, 0, 0, 0, 0, 0, 4, 5],
 [0, 0, 0, 0, 8, 6, 3, 0, 0],
 [3, 0, 0, 0, 0, 5, 0, 9, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 4]]

In [23]:
m, done = solve_matrix(m)
m

[[7, 2, 1, 8, 4, 9, 5, 6, 3],
 [8, 3, 4, 5, 6, 2, 1, 7, 9],
 [5, 6, 9, 3, 7, 1, 4, 8, 2],
 [4, 9, 3, 6, 5, 8, 7, 2, 1],
 [1, 5, 7, 2, 9, 4, 8, 3, 6],
 [6, 8, 2, 1, 3, 7, 9, 4, 5],
 [2, 4, 5, 9, 8, 6, 3, 1, 7],
 [3, 7, 6, 4, 1, 5, 2, 9, 8],
 [9, 1, 8, 7, 2, 3, 6, 5, 4]]

In [24]:
done

True

In [25]:
import random

In [26]:
def generate_grid(qty, seed):
    random.seed(seed)
    mat = [([0]*9) for i in range(9)]

    for nb in range(qty):
        valid = False
        while not valid:
            x = random.randint(0, 8)
            y = random.randint(0, 8)
            if mat[x][y] == 0:
                try:
                    mat[x][y] = random.choice(simple_check(mat, x, y))
                    valid = True
                except:
                    valid = False
    return mat

In [27]:
m = generate_grid(17, 42)
m

[[0, 0, 0, 0, 0, 0, 0, 6, 0],
 [5, 0, 0, 4, 0, 0, 1, 0, 2],
 [0, 0, 0, 9, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 0, 0, 7, 6],
 [3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 9, 0, 0, 0, 0],
 [1, 5, 0, 0, 0, 6, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [7, 0, 0, 0, 8, 0, 0, 0, 0]]

In [35]:
def generate_valid_grid(qty):
    done = False
    seed = 0
    while not done:
        mat = generate_grid(qty, seed)
        solved_mat, done = solve_matrix(mat)
        seed += 1
    return mat, seed

m, i = generate_valid_grid(35)

In [36]:
m

[[0, 0, 6, 0, 0, 0, 7, 0, 5],
 [0, 7, 9, 5, 2, 4, 0, 6, 0],
 [8, 0, 3, 0, 0, 1, 0, 9, 4],
 [5, 3, 0, 4, 9, 6, 0, 2, 0],
 [7, 6, 1, 0, 0, 0, 0, 4, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 2, 0, 0, 0, 0, 4, 0, 0],
 [3, 0, 0, 1, 0, 2, 8, 0, 0],
 [0, 0, 0, 0, 4, 8, 0, 0, 2]]

In [37]:
solve_matrix(m)

([[2, 4, 6, 8, 3, 9, 7, 1, 5],
  [1, 7, 9, 5, 2, 4, 3, 6, 8],
  [8, 5, 3, 6, 7, 1, 2, 9, 4],
  [5, 3, 8, 4, 9, 6, 1, 2, 7],
  [7, 6, 1, 2, 8, 3, 5, 4, 9],
  [4, 9, 2, 7, 1, 5, 6, 8, 3],
  [6, 2, 8, 9, 5, 7, 4, 3, 1],
  [3, 5, 4, 1, 6, 2, 8, 7, 9],
  [9, 1, 7, 3, 4, 8, 6, 5, 2]],
 True)